In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
class Data():
    def __init__(self, path, input_features, output_features):
        self.path = path
        self.inp = input_features
        self.out = output_features

    def extract(self):
        df = pd.read_csv(self.path)
        X = df[self.inp]
        y = df[self.out]
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.35, shuffle=False)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.15, shuffle=False)
        X_train = torch.tensor(X_train.values, dtype=torch.float32)
        y_train = torch.tensor(y_train.values, dtype=torch.float32)
        X_test = torch.tensor(X_test.values, dtype=torch.float32)
        y_test = torch.tensor(y_test.values, dtype=torch.float32)
        X_val = torch.tensor(X_val.values, dtype=torch.float32)
        y_val = torch.tensor(y_val.values, dtype=torch.float32)
        return X_train, X_test, X_val, y_train, y_test, y_val


In [4]:
input_columns = ['GaH_V', 'GaL_V', 'GbH_V', 'GbL_V', 'GcH_V', 'GcL_V', 'speed_radps'] #['mAngle_rad'] # ,
output_columns = ['Ia_amps', 'Ib_amps', 'Ic_amps']
data = Data('PMSM.csv', input_columns, output_columns)
train_in, test_in, val_in, train_out, test_out, val_out = data.extract()

In [5]:
max_value_last_column, _ = torch.max(train_in[:, 6], dim=0)
train_in[:, 6] /= max_value_last_column.item()

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model_conti = nn.Sequential(
              nn.Conv1d(1, 256, kernel_size=3, padding=1),
              nn.Tanh(),
              nn.Conv1d(256, 1024, kernel_size=3, padding=1),
              nn.ReLU()
          )
        self.model_bin = nn.Sequential(
              nn.Linear(6, 1024),
              nn.ReLU()
          )
        self.out = nn.Linear(2048, 3)

    def forward(self, x_bin, x_conti):
        conti_preds = self.model_conti(x_conti)
        bin_preds = self.model_bin(x_bin)
        comb = torch.cat((conti_preds.squeeze(2), bin_preds), dim=1)
        out = self.out(comb)
        return out

In [7]:
class Motor(nn.Module):
    def __init__(self, train_in, train_out, val_in, val_out, test_in, test_out, criterion, batch_size):
        super(Motor, self).__init__()
        dataset = TensorDataset(train_in[:, :6], train_in[:, -1].unsqueeze(1), train_out)
        train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        # dataset = TensorDataset(val_in, val_out)
        val_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        self.trainloader = train_dataloader
        self.valloader = val_dataloader
        # self.val_in = val_in
        # self.val_out = val_out
        self.test_in = test_in
        self.test_out = test_out
        self.criterion = criterion

    def train(self, epochs, optimizer, model):
        for epoch in range(epochs):
            train_loss = 0
            for X_bin, X_conti, y_batch in self.trainloader:
              optimizer.zero_grad()
              # predictions = model(X_batch)
              predictions = model(X_bin, X_conti.unsqueeze(1))
              loss = self.criterion(predictions, y_batch)
              loss.backward()
              optimizer.step()
              train_loss += loss.item()
              # if(epoch==4):
              #   print(predictions)
              #   print(y_batch)
            # preds = model(self.train_in)
            # train_loss = torch.sum(self.criterion(preds, self.train_out))
            print(f'Epoch{epoch}: Train loss = {train_loss/len(self.trainloader)}')
        return model

    def test(self, model):
        preds = model(self.test_in)
        test_loss = torch.sum(self.criterion(preds, self.test_out))
        print(f'Test loss = {test_loss}')

In [8]:
criterion = nn.MSELoss()
motor = Motor(train_in, train_out, val_in, val_out, test_in, test_out, criterion, 32)
model = Model()
optimizer = torch.optim.Adam(params=model.parameters(), lr = 0.001, weight_decay=1e-5)
model = motor.train(10, optimizer, model)
# motor.test(model)

Epoch0: Train loss = 1.570969114453611
Epoch1: Train loss = 1.6501222185680062
Epoch2: Train loss = 1.7098595175365674
Epoch3: Train loss = 1.8204793953539546
Epoch4: Train loss = 1.7375896686385994
Epoch5: Train loss = 1.830011630370834
Epoch6: Train loss = 1.7247578925355496
Epoch7: Train loss = 1.838897946892399


KeyboardInterrupt: 

predict control signals and angle, predict currents

In [9]:
max_value_last_column, _ = torch.max(test_in[:, 6], dim=0)
test_in[:, 6] /= max_value_last_column.item()

In [11]:
pred = model(test_in[0][:6].unsqueeze(0), test_in[0][6].unsqueeze(0).unsqueeze(0).unsqueeze(1))

In [12]:
pred

tensor([[ 318.8877,  292.7519, -661.6600]], grad_fn=<AddmmBackward0>)

In [13]:
o = test_out[0]
o

tensor([ 246.8394, -281.8730,   35.0336])

In [14]:
nn.MSELoss()(pred, o)

/home/srinivasan/miniconda3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(273588.9062, grad_fn=<MseLossBackward0>)